# Homework Assignment Notebook - Quantum Variables & Functions

This notebook, containing all required deliverables, should be submitted for the introduction to quantum programming session homework assignment.

## Your Assignment:

After reviewing the [quantum variables and functions tutorial](https://docs.classiq.io/latest/classiq_101/classiq_concepts/design/quantum_variables_and_functions/), use the Classiq Python SDK with your preferred platform to:

1. Design a quantum algorithm that prepares two qubits in the superposed state:
\begin{equation}
\frac{1}{\sqrt{2}}|0\rangle |0\rangle+ \frac{1}{2}|1\rangle\big(|0\rangle+i |1\rangle \big) 
\end{equation}
To achieve this, create a function that uses the built-in `control` operators to apply rotations on a target qubit, conditioned on a control qubit. In your `main` function, declare and initialize the two required qubits as variables of your choice before invoking your function. 
Finally, execute your algorithm on the Classiq state vector simulator and verify that the final state matches the required superposition.

    Note: do not use the controlled rotation built-in functions

2. Design a quantum algorithm for approximating the cosine function ($cos(x)$) in the range $x\in[0,1)$.
   For this task, initiate $x$ as a `QNum` variable of $3$ qubits, prepared in a superposition of the values:
   \begin{equation}
   x\in \{\frac{k}{8}: k=0,1,\ldots,7\}
   \end{equation}
Hint: Think about how you could recreate the behavior of the cosine function using only basic arithmetic operations.

   Aim for the highest accuracy possible. You can use the code in the “Your Solution” section to evaluate the accuracy of your approximation by comparing it with the classical computation of $cos(x)$ (through NumPy).




Your code and explanations should be included in the following section, which provides a step-by-step outline of what needs to be submitted.

## Your Solution:

Follow the instructions in the #TODO comments in each snippet and insert your code to ensure the algorithms run correctly and produce the desired outcomes.

1. Manual state preparation:

To generate the required state, begin by defining a function that applies controlled operations to the target qubit. Note that the current function is just a placeholder allowing the rest of the code to run; update its signature and contents to produce the desired state.

In [ ]:
from classiq import *
import numpy as np
import json

@qfunc
def your_function(array: QArray[QBit]): # TODO: this is a placeholder signature so that the code in the notebook runs! 
                                        # Change the signature and name of the function to your signature and name
    pass # TODO: this is a placeholder so that the code in the notebook runs! Delete the pass and add your contents!


Now, build your `main` function. Modify its signature and contents to declare, initialize, and (if necessary) prepare the required variables, before invoking the function defined earlier:

In [ ]:
@qfunc
def main(array: Output[QArray[QBit]]): # TODO: this is a placeholder signature so that the code in the notebook runs! 
                                       # Change the signature and name of the function to your signature and name  
    allocate(2,array)   # TODO: this is a placeholder operation so that the code in the notebook runs! Change the contents to your contents!
    your_function(array) # TODO: this is a placeholder operation so that the code in the notebook runs! Change the contents to your contents!
    
# Creating the quantum model
qmod = create_model(main)

Use the following code to synthesize the algorithm into a quantum program using the Classiq synthesis engine, setting the execution preferences to the Classiq state vector simulator:

In [ ]:
# from classiq import (ClassiqBackendPreferences,ClassiqSimulatorBackendNames, ExecutionPreferences)

# Defining the backend as the Classiq state vector simulator
state_vector=ClassiqBackendPreferences(
        backend_name=ClassiqSimulatorBackendNames.SIMULATOR_STATEVECTOR)

# Setting the execution preferences
qmod = set_execution_preferences(qmod, ExecutionPreferences(backend_preferences=state_vector))

# Synthesizing the model into a quantum program
qprog= synthesize(qmod)
show(qprog)

Finally, run the following code that executes the program on the Classiq simulator and displays the measurement outcomes along with their relative phases. Confirm that the results match your expectations.



In [ ]:
import math
from fractions import Fraction

# Executing the quantum program
job = execute(qprog)

# Visualizing as a diagram in the IDE
job.open_in_ide()

# Retrieving the results as a nested data structure
res = job.get_sample_result()

# Extracting counts and state vector
counts = res.counts
state_vector = res.state_vector

# Specify your desired key order
raw_keys = ["00", "01", "10", "11"]

# Normalize the counts by the total number of shots to receive probabilities
probabilities = {
    key: counts.get(key, 0) / 2048 for key in raw_keys
}

# Determine the global phase from the "00" key (if present)
ref_val = state_vector.get("00", 0j)
ref_phase = math.atan2(ref_val.imag, ref_val.real)

# Build the phase dictionary in a single pass
phase_dict = {}
for key in raw_keys:
    val = state_vector.get(key, 0j)
    angle = math.atan2(val.imag, val.real) - ref_phase  # subtract global phase

    # Convert to fraction of π
    frac = Fraction(angle / math.pi).limit_denominator(100)

    # Format fraction as a string
    if frac == 0:
        phase_str = "0"
    elif frac.denominator == 1:
        if frac.numerator == 1:
            phase_str = "π"
        elif frac.numerator == -1:
            phase_str = "-π"
        else:
            phase_str = f"{frac.numerator}π"
    else:
        phase_str = f"{frac.numerator}π/{frac.denominator}"
    
    phase_dict[key] = phase_str

print("Probabilities =", probabilities)
print("Relative phases =", phase_dict)

When running the above code, a new window will open and direct you to the IDE, allowing you to execute the circuit and observe the resulting diagram, complete with amplitudes and phases of the various states. Keep in mind that the phases are correct only up to a global phase, so only relative phases should be considered

2. The approximation of the cosine function should be implemented within the following `compute_sin` function. Keep its name and variables unchanged, and fill in the necessary content.

In [ ]:
@qfunc
def compute_cos(x: QNum, cos_x: Output[QNum]): # Do not change the variables declared in the signature and the name of function
    allocate(3,cos_x) # TODO delete the content and write your own code contents 
    pass # TODO delete pass and write your own code contents

You can add further variables to the function, but make sure to take them into account when you call the function.  

Next, edit the `main` function below. Retain the same variables and declaration parameters, adding any additional variables as required, and include any necessary preparation steps before calling the `compute_cos` function: 

In [ ]:
@qfunc
def main(x: Output[QNum[3,UNSIGNED,3]],cos_x: Output[QNum]): # Keep variable declarations, and add variables as needed.
    allocate(3,x)
    hadamard_transform(x)
    #TODO add any other preparational steps aS needed
    compute_cos(x,cos_x)
    
# Generating the model
qmod = create_model(main)


Synthesize and execute the circuit on the Classiq simulator (supporting up to 25 qubits) using the following code, outputting ordered pairs of the different $x$ and corresponding approximated $cos(x)$ values. 

In [ ]:
# Setting the constraints (up to 25 qubits) 
constraints = Constraints(max_width=25)
qmod_with_constraints = set_constraints(qmod,constraints)
#synthesizing the model to a quantum program
qprog=synthesize(qmod_with_constraints)
show(qprog)

# Executing the quantum program
job = execute(qprog)                                                                    
res = job.get_sample_result() # A nested data structure storing the results

# Extracting (x, cos_x) pairs from the parsed states and sorting them by x
ordered_pairs = [(entry["x"], entry["cos_x"]) for entry in res.parsed_states.values()]
ordered_pairs.sort(key=lambda pair: pair[0])
print("(x, cos_x):", ordered_pairs)

Verify that the results make sense before continuing to evaluate the approximation accuracy.

Finally, use the following code to evaluate your quantum algorithm’s approximation accuracy by comparing its results to classical (NumPy-based) cosine values over the same domain, utilizing the maximal distance as the metric.

In [ ]:
import numpy as np

def evaluate_score_prec(res):
    """
    Evaluate the score based on the provided result object `res` and user-specified precision.

    Parameters:
        res: The result object containing `parsed_states` and other information.
        user_input_precision: The precision level for the evaluation.

    Returns:
        max_distance: The maximum distance between expected and measured y values.
    """
    # Constants:
    calculated_precision = 3 # number of qubits used for the QNum variable x
    domain = np.arange(0, 1, 1 / 2**calculated_precision)
    expected_y = np.cos(domain) # Calculates the cos values classically with NumPy

    # Extract parsed states
    parsed_states = res.parsed_states
    
    # Sort parsed states by 'x' values
    parsed_counts = sorted(parsed_states.items(), key=lambda item: item[1]['x'])
    
    # Form the dictionary with x and y values
    results_dict = {float(s[1]['x']): float(s[1]['cos_x']) for s in parsed_counts}

    # Verify all strings were sampled, also no superpositions
    assert len(results_dict) == 2 ** calculated_precision, \
        f"Expected {2 ** calculated_precision} unique states, but got {len(results_dict)}."

    # Compare user's results against expected results:
    measured_y = []
    for x_val in domain:
        # Find the floored x value that matches user precision
        x_val_floored = int(x_val * (2 ** calculated_precision)) / (2 ** calculated_precision)
        measured_y.append(results_dict.get(x_val_floored, 0))  # Default to 0 if key is missing

    # Calculate the maximal distance metric:
    max_distance = np.max(np.abs(expected_y - np.array(measured_y)))
    return max_distance

In [ ]:
max_distance = evaluate_score_prec(res)
print(f"Maximal Distance: {max_distance}")

If you have followed the steps in this section and every code snippet runs successfully, congratulations! you’re done.
Please upload this notebook via the submission form sent to you.